# Tarea 02: idf

### En esta tarea, se trabajará con un corpus de textos para obtener las palabras mas y menos relevantes a partir de un análisis por tf-idf. Deberás seguir los pasos de este notebook

In [34]:
"""
Paso 1) 
Del archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title", 
y generar un nuevo archivo que SOLO contenga dichas preguntas.
NOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y 
generar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación
y se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura

Paso 2) 
Traducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de 
traducción automático utilizando Google Sheets y Google Translate)
Se deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las 
traducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)

Paso 3) 
Con este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar 
OBLIGATORIAMENTE los siguientes preprocesamientos:
- Lematización de todas las palabras
- Filtrado de StopWords
- Pasar todo a minúsculas

Paso 4) 
Después de haber limpiado el dataset anterior, generar el vector de idf correspondiente a TODOS los textos
y mostrarlo en pantalla

Paso 5) Regresar al Dataset original, y remover todas aquellas palabras que contengan un valor de idf menor
al promedio de TODOS los idfs de la tabla obtenida

Paso 6)
Generar nuevamente la tabla de idf a partir de valores de idf para los textos filtrados y mostrarla en pantalla

Paso 7) Imprimir en pantalla el top de las 10 palabras MAS relevantes, y el top de las 20 palabras MENOS 
relevantes

NOTA: Recuerda que deberás de entregar 2 archivos, un .csv con los 2,000 textos originales y sus traducciones
y un .ipynb con todo el procedimiento realizado con sus respectivos comentarios, y DocStrings
IMPORTANTE: Todo el proceso deberá realizarse por medio de métodos, NO se aceptará programación estructurada,
por lo que, por ejemplo, deberá haber un método para filtrar StopWords, otro para obtener el promedio de 
idf de todo el conjunto de palabras, etc. 
"""

'\nPaso 1) \nDel archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title", \ny generar un nuevo archivo que SOLO contenga dichas preguntas.\nNOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y \ngenerar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación\ny se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura\n\nPaso 2) \nTraducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de \ntraducción automático utilizando Google Sheets y Google Translate)\nSe deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las \ntraducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)\n\nPaso 3) \nCon este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar \nOBLIGATORI

In [35]:
# Recuerda que todos los métodos que utilices deberán de contar con el formato DocString
# como en el ejemplo que se muestra a continuación:
# Ejemplo de formato Docstrings:

# def NombreFuncion(arg1, arg2, arg3):
#     """
#     Este método sirve para... utilizando... y devuelve...
    
#     Args:
#         string arg1: Esta es una cadena de texto que...
#         int arg 2: Es un número entero que se usa para...
#         dict arg 3: Diccionario que sirve para...

#     Returns:
#         string: Cadena del texto ya corregido...
#         int: El la cantidad de correcciones realizadas...
#     """

#     # Aquí debe de ir la lógica de la función (Después de la documentación)
#     Texto = ""
#     corr = 5
    
#   return Texto, corr

In [36]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import re

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")


def extract_and_save_titles(input_file, output_file, num_rows=2000):
    """
    Extract a specified number of rows from the "Title" column of a CSV file and save to a new CSV file.

    Parameters:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the extracted titles as a new CSV file.
        num_rows (int, optional): Number of rows to extract from the "Title" column. Default is 2000.
    """
    df = pd.read_csv(input_file, encoding="latin1")
    df_titles = pd.DataFrame(df.head(num_rows)["Title"])
    df_titles.to_csv(output_file, index=False)


class TextAnalysis:
    """
    Class to analyze text data.

    Parameters:
        filepath (str): Path to the input CSV file.
    """
    def __init__(self, filepath, flag):
        """
        Initializes the TextAnalysis class with given file path, and preprocesses the data.

        Parameters:
            filepath (str): Path to the input CSV file.
        
        Returns:
            Nothing. Initializes attributes and preprocesses the data.
        """
        self.filepath = filepath
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = nltk.corpus.stopwords.words("spanish")
        self.vectorizer = CountVectorizer()
        self.read_data()
        self.preprocess_translated_text()
        self.calculate_idf()
        if flag:
            self.filter_below_average_idf()

    def read_data(self):
        """
        Reads the CSV data from the provided file path and saves the 'Translated_Texts' column to the class.

        Returns:
            Nothing. Saves the 'Translated_Texts' to the class.
        """
        self.df = pd.read_csv(self.filepath)["Translated_Texts"]
        self.df = self.df.dropna()

    def preprocess_translated_text(self):
        """
        Tokenizes, lemmatizes, and removes stopwords from the 'Translated_Texts' column.

        Returns:
            Nothing. Updates the 'Translated_Texts' column in the dataframe with the preprocessed text.
        """
        def process(text):
            words = nltk.word_tokenize(text)
            words = [
                self.lemmatizer.lemmatize(word.lower())
                for word in words
                if word.lower() not in self.stop_words and not (any(c.isalpha() for c in word) and any(c.isdigit() for c in word))
            ]
            return " ".join(words)
        self.df = self.df.apply(process)
        self.df = self.df.dropna()

    def calculate_idf(self, data_df=None):
        """
        Calculates the IDF values of the words.

        Parameters:
            data_df (DataFrame, optional): DataFrame to compute IDF values on. 
                If not provided, the class's attribute is used.

        Returns:
            Nothing. Saves the IDF values, the average IDF value, and a dataframe of words with their IDF values to the class.
        """
        if data_df is None:
            data_df = self.df

        tfidf_vectorizer = TfidfVectorizer(use_idf=True)
        tfidf_vectorizer.fit_transform(data_df)
        idf_values = tfidf_vectorizer.idf_
        idf_dict = dict(zip(tfidf_vectorizer.get_feature_names_out(), idf_values))

        self.idf_df = pd.DataFrame(list(idf_dict.items()), columns=["Word", "IDF Value"])
        self.average_idf = np.mean(self.idf_df["IDF Value"])
        self.df_sorted = self.idf_df.sort_values(by='IDF Value', ascending=False)
        self.idf_dict = idf_dict

    def filter_below_average_idf(self):
        """
        Filters out the words with below average IDF values.

        Returns:s
            Nothing. Saves the filtered text to the 'below_average_idf' attribute of the class.
        """
        self.below_average_idf_df = []
        for word, idf in self.idf_dict.items():
            if idf < self.average_idf:
                self.below_average_idf_df.append(word.lower())


    def delete_below_average_idf(self):
        """
        Deletes the words with below average IDF values from the 'Translated_Texts' column.

        Returns:
            Nothing. Saves the filtered df in class and in file.
        """
        def remove_low_idf_words(text):
            tokens = text.split()
            
            tokens = [re.sub(r'^\((.*?)\)$|^[.,¿?!:\']|[.,¿?!:\']$', r'\1', token) for token in tokens]
            
            cleaned_tokens = [token.lower() for token in tokens if token.lower() not in self.below_average_idf_df]
            return ' '.join(cleaned_tokens)

        df = pd.read_csv('TranslatedQuestions.csv', encoding="utf-8")
        df["Translated_Texts"] = df["Translated_Texts"].apply(remove_low_idf_words)
        df["Translated_Texts"].to_csv("CleanedQuestions.csv", index=False, encoding="utf-8")
        self.df_cleaned = df["Translated_Texts"]


    def top_bottom_words(self, top_n=10, bottom_n=20):
        """
        Retrieves the top and bottom words based on their IDF values.

        Parameters:
            top_n (int): Number of top words to retrieve. Default is 10.
            bottom_n (int): Number of bottom words to retrieve. Default is 20.

        Returns:
            tuple: A tuple containing two dataframes. 
            The first dataframe contains the top words and their IDF values. 
            The second dataframe contains the bottom words and their IDF values.
        """
        sorted_idf = sorted(self.idf_dict.items(), key=lambda x: x[1], reverse=True)
        top_words = sorted_idf[:top_n]
        bottom_words = sorted_idf[-bottom_n:]
        df_top_words = pd.DataFrame(top_words, columns=["Word", "IDF Value"])
        df_bottom_words = pd.DataFrame(bottom_words, columns=["Word", "IDF Value"])
        return df_top_words, df_bottom_words


[nltk_data] Downloading package punkt to /home/eubgo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eubgo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eubgo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
if __name__ == "__main__":
      """
      Paso 1)
      Del archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title",
      y generar un nuevo archivo que SOLO contenga dichas preguntas.
      NOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y
      generar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación
      y se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura
      """
      extract_and_save_titles("Questions.csv", "Questions_2000.csv")

      """
      Paso 2) 
      Traducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de 
      traducción automático utilizando Google Sheets y Google Translate)
      Se deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las 
      traducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)
      """
      # In TranslatedQuestions.csv file

      """
      Paso 3) 
      Con este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar 
      OBLIGATORIAMENTE los siguientes preprocesamientos:
      - Lematización de todas las palabras
      - Filtrado de StopWords
      - Pasar todo a minúsculas
      """
      analysis = TextAnalysis("TranslatedQuestions.csv", 1)

      """
      Paso 4) 
      Después de haber limpiado el dataset anterior, generar el vector de idf correspondiente a TODOS los textos
      y mostrarlo en pantalla
      """
      print("Initial IDF Table:")
      print(analysis.df_sorted)

      """
      Paso 5) Regresar al Dataset original, y remover todas aquellas palabras que contengan un valor de idf menor
      al promedio de TODOS los idfs de la tabla obtenidaa
      """
      analysis.filter_below_average_idf()
      analysis.delete_below_average_idf()

      """
      Paso 6)
      Generar nuevamente la tabla de idf a partir de valores de idf para los textos filtrados y mostrarla en pantalla
      """
      new_analysis = TextAnalysis("CleanedQuestions.csv", flag=False) 
      print("\nFiltered IDF Table:")
      print(new_analysis.df_sorted)
      
      """
      Paso 7) Imprimir en pantalla el top de las 10 palabras MAS relevantes, y el top de las 20 palabras MENOS 
      relevantes
      """
      top, bottom = analysis.top_bottom_words()
      print("\nTop 10 Words:")
      print(top)
      print("\nBottom 20 Words:")
      print(bottom)
      

Initial IDF Table:
         Word  IDF Value
0          04   7.908255
1354  fábrica   7.908255
1367     gear   7.908255
1366      gdb   7.908255
1365    gconf   7.908255
...       ...        ...
2389    puedo   4.016435
208   archivo   3.874015
954    django   3.495457
771      cómo   2.315404
2420   python   1.602893

[3205 rows x 2 columns]

Filtered IDF Table:
           Word  IDF Value
0            04   7.874198
1438       owfs   7.874198
1452  parciales   7.874198
1451    parcial   7.874198
1450     parche   7.874198
...         ...        ...
2004       toda   5.734132
2097   variable   5.676974
645          do   5.571613
1612     python   5.040985
2145     window   4.696145

[2194 rows x 2 columns]

Top 10 Words:
   Word  IDF Value
0    04   7.908255
1   100   7.908255
2  1000   7.908255
3   104   7.908255
4  1123   7.908255
5    20   7.908255
6  2006   7.908255
7  2009   7.908255
8    23   7.908255
9   301   7.908255

Bottom 20 Words:
          Word  IDF Value
0     archivos   4